https://github.com/prdeepakbabu/Python/blob/master/Deep%20learning%20gender/Deep%20Learning%20(RNN%20-%20LSTMs)%20Predict%20Gender%20from%20Name.ipynb From this link with different data

https://github.com/mhjabreel/CharCnn_Keras/blob/master/models/char_cnn_zhang.py char CNN

In [1]:
import tensorflow as tf
from tensorflow import keras

2021-10-26 14:31:43.835149: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [2]:
from __future__ import print_function

from sklearn.preprocessing import OneHotEncoder
from tensorflow.python.keras.layers.core import Dense, Activation, Dropout
from tensorflow.python.keras.preprocessing import sequence
from tensorflow.python.keras.models import Sequential, Model, load_model
from tensorflow.python.keras.layers import Dense, Embedding
from tensorflow.python.keras.layers import LSTM
from tensorflow.python.keras.layers import Input, Dense, Flatten
from tensorflow.python.keras.callbacks import TensorBoard
import pandas as pd
import numpy as np
import os
from sklearn.metrics import classification_report

In [3]:
import sys  
sys.path.insert(0, '../PreProcessing/')

from utils import *
from parse_df import *
from split_name import *
from char_utils import *

In [4]:
print(tf.__version__)

2.4.0


In [5]:
physical_devices = tf.config.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)
from tensorflow.python.client import device_lib

def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU' or x.device_type == 'CPU']

2021-10-26 14:31:46.462033: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-10-26 14:31:46.464555: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-10-26 14:31:46.523787: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:3b:00.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2021-10-26 14:31:46.524924: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:d8:00.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2021-10-26 14:31:46.524965: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2

In [6]:
get_available_devices()

2021-10-26 14:32:07.979216: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX512F
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-10-26 14:32:07.984024: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-10-26 14:32:08.268872: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:3b:00.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2021-10-26 14:32:08.269507: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:d8:00.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMem

['/device:CPU:0', '/device:GPU:0', '/device:GPU:1']

### PreProcessing

In [7]:
cbse_df = getCBSEData(False)
cbse_df = cleanDf(cbse_df, 'Name')

Gender
Boy     114088
Girl     91456
Name: Name, dtype: int64


In [8]:
train_x, train_y, val_x, val_y, test_x, test_y = splitData(cbse_df, 0.7, 0, 'Name', 'char_lstm_cbse_oov.txt')

(train:(150851, 5), val:(0, 0), test:(54694, 5))
Train: ((150797, 30),(150797, 2)) Val:((0,),(0,)) Test: ((54683, 30),(54683, 2))


In [9]:
voc = vocab()

### Build Model

In [10]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  

In [11]:
inputs = Input(MAXLEN_NAME_CHAR_LSTM)
x = Embedding(len(voc)+1, len(voc)+1)(inputs)
x = LSTM(512, return_sequences=True, activation='tanh')(x)
x = Dropout(0.2)(x)
x = LSTM(512, return_sequences=False, activation='tanh')(x)
x = Dropout(0.2)(x)
predictions = Dense(2, activation='softmax')(x)
model = Model(inputs=inputs, outputs=predictions)

2021-10-26 14:33:34.938338: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-10-26 14:33:34.939199: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:3b:00.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2021-10-26 14:33:34.939764: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:d8:00.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2021-10-26 14:33:34.939812: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-10-26 14:33:34.939972: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.1

In [12]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 30)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 30, 29)            841       
_________________________________________________________________
lstm (LSTM)                  (None, 30, 512)           1110016   
_________________________________________________________________
dropout (Dropout)            (None, 30, 512)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 2)                 1026  

In [13]:
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [14]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  2


In [16]:
batch_size=1000

In [17]:
model.fit(train_x, train_y,batch_size=batch_size,epochs=10,validation_data=(test_x, test_y))

2021-10-26 14:34:55.485960: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-10-26 14:34:55.505465: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2100000000 Hz


Epoch 1/10


2021-10-26 14:34:59.923706: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2021-10-26 14:35:00.700303: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11


151/151 [==============================] - 91s 567ms/step - loss: 0.6053 - accuracy: 0.6495 - val_loss: 0.5193 - val_accuracy: 0.7533
Epoch 2/10
151/151 [==============================] - 85s 562ms/step - loss: 0.4298 - accuracy: 0.8098 - val_loss: 0.3521 - val_accuracy: 0.8595
Epoch 3/10
151/151 [==============================] - 85s 561ms/step - loss: 0.3055 - accuracy: 0.8821 - val_loss: 0.3008 - val_accuracy: 0.8783
Epoch 4/10
151/151 [==============================] - 85s 561ms/step - loss: 0.2512 - accuracy: 0.9054 - val_loss: 0.2855 - val_accuracy: 0.8837
Epoch 5/10
151/151 [==============================] - 85s 561ms/step - loss: 0.2286 - accuracy: 0.9148 - val_loss: 0.2530 - val_accuracy: 0.9080
Epoch 6/10
151/151 [==============================] - 85s 560ms/step - loss: 0.2017 - accuracy: 0.9265 - val_loss: 0.2442 - val_accuracy: 0.9149
Epoch 7/10
151/151 [==============================] - 84s 560ms/step - loss: 0.1826 - accuracy: 0.9350 - val_loss: 0.2271 - val_accuracy: 0.9

In [18]:
score, acc = model.evaluate(test_x, test_y)
print('Test score:', score)
print('Test accuracy:', acc)

1709/1709 [==============================] - 56s 33ms/step - loss: 0.2112 - accuracy: 0.9242
Test score: 0.2111518532037735
Test accuracy: 0.9242360591888428


In [19]:
filename = "SavedModels/FinalSet/CharLSTM/CBSEModel/CBSEModel"
model.save(filename)

2021-10-26 14:50:39.385974: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: SavedModels/FinalSet/CharLSTM/CBSEModel/CBSEModel/assets


In [20]:
pred = model.predict(np.asarray(test_x))
y_pred = np.argmax(pred, axis=1)
y_true = np.argmax(test_y, axis=1)

target_names = ['Male','Female']

print("------------------RESULTS-------------------------------")
print(classification_report(y_true, y_pred, target_names=target_names))
print("--------------------------------------------------------")

------------------RESULTS-------------------------------
              precision    recall  f1-score   support

        Male       0.93      0.94      0.93     31158
      Female       0.92      0.90      0.91     23525

    accuracy                           0.92     54683
   macro avg       0.92      0.92      0.92     54683
weighted avg       0.92      0.92      0.92     54683

--------------------------------------------------------
